In [35]:
import pandas as pd
import requests as re
from io import BytesIO
from os import path, makedirs
from unidecode import unidecode

# Produção de habitação de interesse social

Vários indicadores do GT Urbanismo são relacionados à produção e entrega de unidades de habitação de interesse social. O formulário 15 é relacionado ao orçamento utilizado na produção de unidades de habitação de interesse social. O primeiro indicador físico está no formulário 18 e diz respeito à meta de provimento de moradias do PdM 2021-2024. O formulário 20 é relacionado a uma iniciativa específica do PdM, mas os dados de execução estão disponíveis apenas no relatório da Função Habitação. Por último, os formulários 19 e 21 são relacionado, mas não possuem meta no PdM, apenas nos ODS da Agenda 2030.

Posteriormente, um novo documento apresentou várias informações diferentes relacionadas ao orçamento da função habitação, onde está incluso o Programa 3002. Portanto, vamos manter todo o orçamento da função habitação e apresentar os diferentes recortes no Qlik Sense.

## ObservaSampa

Como vamos usar vários indicadores do ObservaSampa parece fazer sentido usar uma função genérica para download de indicadores do ObservaSampa.

In [36]:
indicadores_observa = [
    {
        "nome_completo": "11.01.02 Número de famílias beneficiadas por procedimentos de regularização fundiária em núcleos urbanos informais",
        "nome_curto": "indicador_110102",
        "id_observa": 283,
        "nome_coluna_valor": "qtd_familias"
    },

    {
        "nome_completo": "11.01.06 Número de famílias beneficiadas com obras de urbanização de assentamentos precários",
        "nome_curto": "indicador_110106",
        "id_observa": 402,
        "nome_coluna_valor": "qtd_familias"
    }, 

    {
        "nome_completo": "11.01.07 Número estimado de domicílios em favelas",
        "nome_curto": "indicador_110107",
        "id_observa": 288,
        "nome_coluna_valor": "qtd_domicilios"
    },

    {
        "nome_completo": "Número de termos de Permissão de Uso (TPU) emitidos em nome da mulher da familia",
        "nome_curto": "indicador_tpu",
        "id_observa": 662,
        "nome_coluna_valor": "qtd_termos"
    },
    {
        "nome_completo": "Produção de habitação de interesse social",
        "nome_curto": "indicador_producao",
        "id_observa": 285,
        "nome_coluna_valor": "qtd_unidades"
    },
    {
        "nome_completo": "01.05.02 Número de famílias em atendimento habitacional provisório (Auxílio Aluguel) por situação de risco e emergência",
        "nome_curto": "indicador_010502",
        "id_observa": 660,
        "nome_coluna_valor": "qtd_familias"
    },
    {
        "nome_completo": "11.01.08 Total de famílias beneficiárias de Auxílio Aluguel por ano",
        "nome_curto": "indicador_110108",
        "id_observa": 547,
        "nome_coluna_valor": "qtd_familias"
    },
    {
        "nome_completo": "Estimativa de domicílios em favela (%)",
        "nome_curto": "indicador_edf",
        "id_observa": 286,
        "nome_coluna_valor": "qtd_domicilios"
    }
]

In [37]:
def extrair_observa(id_observa:int) -> pd.DataFrame:
    url_observa = f'https://observasampa.prefeitura.sp.gov.br/arquivo.php?cd_indicador={id_observa}'
    df_observa = pd.read_csv(url_observa, sep=';', decimal=',', encoding='utf8')
    return df_observa

In [38]:

def padronizar_subprefeitura(regiao: str) -> str:
    if pd.isna(regiao): return 'Sem informação'
    return unidecode(regiao).strip().lower()

In [39]:
def tratar_indicador_regionalizado(df: pd.DataFrame) -> pd.DataFrame:
    df = df[df['nivel_regional'] == 'Subprefeitura']
    return df

In [40]:
def tratar_indicador_nao_regionalizado(df: pd.DataFrame) -> pd.DataFrame:
    df = df[df['nivel_regional'] == 'Município']
    df = df.drop(columns=['nivel_regional', 'região'])
    return df

In [41]:
def tratar_anos(df: pd.DataFrame,
                nome_coluna_valor:str,
                anos: list[str]=['2022', '2023', '2024']) -> pd.DataFrame:
    
    # Detectar colunas ID dinamicamente
    id_cols = [col for col in ['região', 'indicador'] if col in df.columns]
    
    df = df.melt(
        id_vars=id_cols,
        value_vars=[col for col in df.columns if col in anos],
        var_name='ano',
        value_name=nome_coluna_valor
    )
    return df

In [42]:
def pipeline_completa(indicador: str) -> pd.DataFrame:
    indicador_info = next((item for item in indicadores_observa if item['nome_curto'] == indicador), None)
    
    df_bom = extrair_observa(indicador_info['id_observa'])

    if df_bom['nivel_regional'].str.lower().str.contains('subprefeitura').any():
        df_bom = tratar_indicador_regionalizado(df_bom)
    else:
        df_bom = tratar_indicador_nao_regionalizado(df_bom)

    df_bom = tratar_anos(df_bom, nome_coluna_valor=indicador_info['nome_coluna_valor'])

    return df_bom

### 11.01.02 Número de famílias beneficiadas por procedimentos de regularização fundiária em núcleos urbanos informais


In [43]:

df_110102 = pipeline_completa('indicador_110102')
df_110102.head()

,região,indicador,ano,qtd_familias
0,São Mateus,11.01.02 Número de famílias beneficiadas por p...,2022,0.0
1,Cidade Tiradentes,11.01.02 Número de famílias beneficiadas por p...,2022,0.0
2,Ermelino Matarazzo,11.01.02 Número de famílias beneficiadas por p...,2022,0.0
3,Freguesia-Brasilândia,11.01.02 Número de famílias beneficiadas por p...,2022,0.0
4,Guaianases,11.01.02 Número de famílias beneficiadas por p...,2022,0.0


### 01.05.02 Número de famílias em atendimento habitacional provisório (Auxílio Aluguel) por situação de risco e emergência

In [44]:
df_010502 = pipeline_completa('indicador_010502')
df_010502.head()

,região,indicador,ano,qtd_familias
0,Vila Mariana,01.05.02 Número de famílias em atendimento hab...,2022,0.0
1,Itaim Paulista,01.05.02 Número de famílias em atendimento hab...,2022,20.0
2,Itaquera,01.05.02 Número de famílias em atendimento hab...,2022,2.0
3,Vila Prudente,01.05.02 Número de famílias em atendimento hab...,2022,98.0
4,Perus,01.05.02 Número de famílias em atendimento hab...,2022,95.0


### 11.01.06 Número de famílias beneficiadas com obras de urbanização de assentamentos precários

In [45]:
df_110106 = pipeline_completa('indicador_110106')
df_110106.head()

,região,indicador,ano,qtd_familias
0,Aricanduva-Formosa-Carrão,11.01.06 Número de famílias beneficiadas com o...,2022,0.0
1,Butantã,11.01.06 Número de famílias beneficiadas com o...,2022,930.0
2,Campo Limpo,11.01.06 Número de famílias beneficiadas com o...,2022,60.0
3,Capela Do Socorro,11.01.06 Número de famílias beneficiadas com o...,2022,3590.0
4,Casa Verde-Cachoeirinha,11.01.06 Número de famílias beneficiadas com o...,2022,0.0


### 11.01.07 Número estimado de domicílios em favelas

In [46]:
df_110107 = pipeline_completa('indicador_110107')
df_110107.head()

,região,indicador,ano,qtd_domicilios
0,Ermelino Matarazzo,11.01.07 Número estimado de domicílios em favelas,2022,4423.0
1,Freguesia-Brasilândia,11.01.07 Número estimado de domicílios em favelas,2022,24720.0
2,Guaianases,11.01.07 Número estimado de domicílios em favelas,2022,5054.0
3,Ipiranga,11.01.07 Número estimado de domicílios em favelas,2022,26195.0
4,Itaim Paulista,11.01.07 Número estimado de domicílios em favelas,2022,8064.0


### 11.01.08 Total de famílias beneficiárias de Auxílio Aluguel por ano

In [47]:
df_110108 = pipeline_completa('indicador_110108')
df_110108.head()

,região,indicador,ano,qtd_familias
0,Butantã,11.01.08 Total de famílias beneficiárias de Au...,2022,2229.0
1,Campo Limpo,11.01.08 Total de famílias beneficiárias de Au...,2022,4708.0
2,Ermelino Matarazzo,11.01.08 Total de famílias beneficiárias de Au...,2022,8.0
3,Freguesia-Brasilândia,11.01.08 Total de famílias beneficiárias de Au...,2022,10.0
4,Guaianases,11.01.08 Total de famílias beneficiárias de Au...,2022,26.0


### Produção de habitação de interesse social

In [48]:
df_his = pipeline_completa('indicador_producao')
df_his.head()

,região,indicador,ano,qtd_unidades
0,Cidade Tiradentes,11.01.03 Número de Unidades Habitacionais entr...,2022,0.0
1,Ermelino Matarazzo,11.01.03 Número de Unidades Habitacionais entr...,2022,0.0
2,Parelheiros,11.01.03 Número de Unidades Habitacionais entr...,2022,0.0
3,Penha,11.01.03 Número de Unidades Habitacionais entr...,2022,0.0
4,Perus,11.01.03 Número de Unidades Habitacionais entr...,2022,48.0


O indicador do formulário 19 não está ligado diretamente a nenhuma meta específica do PdM, mas está em consonância com o indicador 11.01.03 dos ODS da Agenda 2030. Os dados desse indicador estão disponíveis no [Observasampa](https://observasampa.prefeitura.sp.gov.br/).

### Número de termos de Permissão de Uso (TPU) emitidos em nome da mulher da familia

Esse indicador possui é um exemplo da transversalidade de gênero, que integra a perspectiva de gênero na construção da politica pública voltada à provisão de habitação de interesse social. Os dados desse indicador estão disponíveis, de maneira não regionalizada, no [Observasampa](https://observasampa.prefeitura.sp.gov.br/), no indicador 05.0a.04. Posteriormente, os dados regionalizados estarão presentes no relatório da Função Habitação.

In [49]:
df_tpu = pipeline_completa('indicador_tpu')
df_tpu.head()

,indicador,ano,qtd_termos
0,05.0a.04 Número de termos de Permissão de Uso ...,2022,293.0
1,05.0a.04 Número de termos de Permissão de Uso ...,2023,384.0
2,05.0a.04 Número de termos de Permissão de Uso ...,2024,458.0


### Estimativa de domicílios em favela (%)

In [50]:
df_edf = pipeline_completa('indicador_edf')
df_edf.head()

,região,indicador,ano,qtd_domicilios
0,Parelheiros,Estimativa de domicílios em favela (%),2022,7.6607
1,São Mateus,Estimativa de domicílios em favela (%),2022,7.4612
2,Penha,Estimativa de domicílios em favela (%),2022,6.4167
3,Pinheiros,Estimativa de domicílios em favela (%),2022,0.2640
4,Pirituba-Jaraguá,Estimativa de domicílios em favela (%),2022,10.6484


# Exportando os arquivos

Neste notebook, vamos apenas salvar os arquivos extraídos na pasta de entrada de dados.

In [51]:
output_dir = path.join('dados', 'urbanismo')
if not path.exists(output_dir):
    makedirs(output_dir)

for nome, df in [
    ('his_entregue_original', df_his),
    ('tpu_emitido_original', df_tpu),
    ('indicador_110107', df_110107),
    ('indicador_110102', df_110102),
    ('indicador_110106', df_110106),
    ('indicador_010502', df_010502),
    ('indicador_110108', df_110108),
    ('indicador_estimativa_de_domicilios_em_favela', df_edf),
]:
    filename = path.join(output_dir, nome)
    df.to_csv(f'{filename}.csv', sep=';', decimal=',', encoding='utf8', index=False)